# Cloud Inpainting on SEN3 Data
This notebook is a quick test to load, mask with clouds and then try and recover the data hidden by clouds using an inpainting DL model

In [1]:
import utils.prepare_dataset as predat
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from network.model import GenModel
from network.loss import lossL1


2024-06-19 07:11:05.962251: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-19 07:11:05.962779: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-19 07:11:05.966510: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-19 07:11:06.014687: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 07:11:06.883071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [ ]:
ds,dsc = predat.prepare_dataset(low_bound=0, high_bound=15, num_clouds=1000, num_samples=1000)

In [ ]:
for i in range(1,13):
    plt.subplot(4,6,2*i-1)
    plt.imshow(ds["Oa01_reflectance"][i])
    plt.subplot(4,6,2*i)
    plt.imshow(dsc["Oa01_reflectance"][i])

## Extracting the bands from the Xarrays

The goal here is to convert the data into a more tensorflow friendly data format

In [ ]:
dsnp = np.ndarray((256,256,11,1000))
dsnp_c = np.ndarray((256,256,11,1000))
for i in range(1,12):
    dsnp[:,:,i-1,:] = np.moveaxis(ds["Oa"+str(i).zfill(2)+"_reflectance"].values,0,2)
    dsnp_c[:,:,i-1,:] = np.moveaxis(dsc["Oa"+str(i).zfill(2)+"_reflectance"].values,0,2)


np.save('Ground_truth.npy', dsnp)
np.save('Cloudy_DS.npy', dsnp_c)

In [2]:
dsnp_c = np.load("Cloudy_DS.npy")
dsnp = np.load("Ground_truth.npy")

In [3]:
#Split into training and test set

train_ds = tf.data.Dataset.from_tensor_slices((dsnp_c[:,:,:,:800], dsnp[:,:,:,:800]))
test_ds = tf.data.Dataset.from_tensor_slices((dsnp_c[:,:,:,800:], dsnp[:,:,:,800:]))

In [4]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

In [7]:
model = GenModel()
coarseNet = model.coarseNet
coarseNet.compile(optimizer='adam', loss=lossL1, metrics=['mean_absolute_error'])
print(coarseNet.summary())


/home/etienne/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:1295: UserWarning: Layer 'gated_conv2d_17' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: '''_TupleWrapper' object is not callable''
  warnings.warn(
/home/etienne/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'gated_conv2d_17', however the layer does not have a `build()` method implemented and it looks like it has unbuilt

TypeError: Exception encountered when calling GatedConv2D.call().

[1m'_TupleWrapper' object is not callable[0m

Arguments received by GatedConv2D.call():
  • args=('<KerasTensor shape=(None, 256, 256, 11), dtype=float32, sparse=None, name=keras_tensor_1>',)
  • kwargs=<class 'inspect._empty'>

In [61]:
coarseNet.fit(train_ds, epochs=1)

/home/etienne/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:1295: UserWarning: Layer 'gated_conv2d_102' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: '''_TupleWrapper' object is not callable''
  warnings.warn(
/home/etienne/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'gated_conv2d_102', however the layer does not have a `build()` method implemented and it looks like it has unbui

TypeError: Exception encountered when calling GatedConv2D.call().

[1m'_TupleWrapper' object is not callable[0m

Arguments received by GatedConv2D.call():
  • args=('<KerasTensor shape=(None, 256, 11, 800), dtype=float32, sparse=None, name=keras_tensor_4>',)
  • kwargs=<class 'inspect._empty'>